Name: Rocky the Bot model
    
    
Description: ML model for prediction of presuasive level for user supplied statements
    
Data: Collected via an online survey
      Currently, only a sample but more data due soon via qualtrics.
        


In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, make_scorer
from sklearn.metrics import roc_curve, auc

In [2]:
# Read in data
df = pd.read_csv('C:\\rocky_bot_model\\static\\rocky_data_121219.csv')

In [3]:
df.shape

(964, 52)

In [4]:
df_input = df.loc[:, 'age':'autonomy05']

In [5]:
df_target = df['score_binary']

In [ ]:
# Original full scale parameter search
params_grid = {'bootstrap': [True, False],
                'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
                'max_features': ['auto'],
                 'min_samples_leaf': [1, 2, 4],
                 'min_samples_split': [2, 5, 10],
                 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
                }
    
    
def get_specificity(y_true, y_pred):
    error = (confusion_matrix(y_true, y_pred)[0,0] / 
            (confusion_matrix(y_true, y_pred)[0,0] + confusion_matrix(y_true, y_pred)[0,1]))
    return error
    
def get_auc(y_true, y_pred):
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    return roc_auc

scoring = {'accuracy': 'accuracy',
           'precision': 'precision',
           'recall': 'recall',
           'specificity': make_scorer(get_specificity, greater_is_better=True),
           'auc': make_scorer(get_auc, greater_is_better=True)
          }

estimator = RandomForestClassifier(random_state=42)
clf = GridSearchCV(estimator, params_grid, cv=10, 
                         scoring=scoring, n_jobs=-1,
                         refit=False
                        )
# Train the classifier
clf.fit(df_input, df_target) 
    
results = pd.DataFrame(clf.cv_results_)

In [ ]:
df_target.value_counts()

In [60]:
results.sort_values(by='mean_test_auc', ascending=False)[['mean_test_auc', 'mean_test_accuracy',
                                                          'mean_test_recall', 'mean_test_specificity'
                                                         ]]

,mean_test_auc,mean_test_accuracy,mean_test_recall,mean_test_specificity
239,0.458896,0.495851,0.669465,0.248327
757,0.458896,0.495851,0.669465,0.248327
129,0.458896,0.495851,0.669465,0.248327
128,0.458896,0.495851,0.669465,0.248327
127,0.458896,0.495851,0.669465,0.248327
667,0.458896,0.495851,0.669465,0.248327
668,0.458896,0.495851,0.669465,0.248327
669,0.458896,0.495851,0.669465,0.248327
677,0.458896,0.495851,0.669465,0.248327
678,0.458896,0.495851,0.669465,0.248327
